In [ ]:
import numpy as np
import pandas as pd

In [ ]:
## ASSOCIATION RULES

## ASSOCIATION RULES

In [ ]:
import pandas as pd
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth


dataset = [['Milk', 'Onion', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Nutmeg', 'Apple', 'Kidney Beans'],
           ['Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs'],
           ['Dill', 'Onion', 'Nutmeg',  'Eggs', 'Yogurt','Kidney Beans'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans',  'Eggs', 'Ice cream'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Kidney Beans', 'Eggs', 'Milk', 'Unicorn', 'Corn'],
           ['Milk', 'Kidney Beans', 'Eggs', 'Apple'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs'],
           [ 'Kidney Beans', 'Yogurt',  'Eggs', 'Apple']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(df, min_support=0.6, use_colnames=True)
frequent_itemsets

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

In [ ]:
rules

In [ ]:
rules.pivot('antecedents', 'consequents', 'antecedent support').fillna(0)

In [ ]:
from collections import Counter 


pair = lambda arr: [(i, v) for i, v in zip(arr[:-1], arr[1:])]
unikeys = lambda _dcc: Counter(_dcc.keys())

def _count_occurences(_sequences):
    """ Given list of lists [[A,B,A,B,A], [C,B,C]] 
        Let Unique Inner-array-wise Pair be UIP
        e.g. (A,B), (B,A), (B,C), (C,B) bot NOT (A,C)
        > count of UIPs in whole list
        > occurence of UIPs in unique lists
        ========================================
        {
        (A, B): 2, 
        (B, A): 2, 
        (C, B): 1, 
        (B, C): 1}
        --- AND --- 
        {
        (A, B): 1, 
        (B, A): 1, 
        (C, B): 1, 
        (B, C): 1}

    """
    cc = Counter(pair(_sequences[0]))
    ucc = unikeys(cc)
    for cX in _sequences[1:]:
        _cc = Counter(pair(cX))

        ucc += unikeys(_cc)
        cc += _cc
    return cc, ucc



def _pair_count_occurence(_sequences):
    _cnt, _occ = _count_occurences(_sequences)
    _cntDF = pd.DataFrame(_cnt, index=['CNT']).T.reset_index()
    _occDF = pd.DataFrame(_occ, index=['OCC']).T.reset_index()

    _cdf = pd.merge(_cntDF, _occDF, how='left',
                    left_on=['level_0', 'level_1'],
                    right_on=['level_0', 'level_1'])

    _cdf.rename(columns={'level_0': 'POSTERIOR',
                         'level_1': 'PRIOR'}, inplace=True)

    _cdf['RELATIVE_OCC'] = _cdf['OCC'].apply(lambda c: c / len(_sequences))
    _cdf['RELATIVE_PRIOR'] = _cdf.apply(lambda r: r['CNT'] / (_cdf[_cdf.PRIOR == r['PRIOR']]['CNT'].sum()), axis=1)
    _cdf['RELATIVE_POSTERIOR'] = _cdf.apply(lambda r: r['CNT'] / (_cdf[_cdf.POSTERIOR == r['POSTERIOR']]['CNT'].sum()),
                                            axis=1)
    _cdf['RELATIVE_CNT'] = _cdf.apply(lambda r: r['CNT'] / (_cdf['CNT'].sum()), axis=1)
    _cdf['RELATIVE_PRIOR_BY_OCC'] = 2*_cdf['RELATIVE_PRIOR']*_cdf['RELATIVE_OCC']/(_cdf['RELATIVE_PRIOR']+_cdf['RELATIVE_OCC'])
    _cdf['RELATIVE_POSTERIOR_BY_OCC'] = 2*_cdf['RELATIVE_POSTERIOR']*_cdf['RELATIVE_OCC']/(_cdf['RELATIVE_POSTERIOR']+_cdf['RELATIVE_OCC'])
    
    return _cdf[
        ['PRIOR', 'POSTERIOR', 'CNT', 'RELATIVE_CNT', 'RELATIVE_PRIOR', 'RELATIVE_PRIOR_BY_OCC', 'RELATIVE_POSTERIOR', 'RELATIVE_POSTERIOR_BY_OCC', 'OCC', 'RELATIVE_OCC']]


In [ ]:
res = _pair_count_occurence(dataset).sort_values('CNT', ascending = False)

In [ ]:
res.sort_values('CNT', ascending = False).head()

In [ ]:
res.pivot('PRIOR', 'POSTERIOR', 'RELATIVE_PRIOR').fillna(0)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

from matplotlib import pyplot as plt

upper_str = lambda s: s.upper()


def _plot_path(_cnt, _item, _col: str = 'CNT'):
    _Ccnt = _cnt[(_cnt['PRIOR'] == _item) | (_cnt['POSTERIOR'] == _item)]
    uniques = dict(enumerate(list(set(_Ccnt['PRIOR']).union(set(_Ccnt['POSTERIOR'])))))
    iuniques = {v: k for k, v in uniques.items()}
    _perc = _Ccnt[_col].dtype != int
    _labels = list(map(upper_str, uniques.values()))

    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=_labels,
            color=list(map(lambda r: 'darkred' if r == _item else 'blue', uniques.values())),
            customdata=_labels, hovertemplate='%{customdata}'

        ),
        link=dict(
            source=list(map(lambda r: iuniques.get(r), _Ccnt.PRIOR.values)),
            target=list(map(lambda r: iuniques.get(r), _Ccnt.POSTERIOR.values)),
            value=list(map(lambda r: 100 * r if _perc else r, _Ccnt[_col].values)),
            color='rgba(200, 205, 206, 0.6)',
            hovertemplate='%{value} of %{source.customdata} to %{target.customdata}'
        ))])

    fig.update_layout(title_text=f"ITEM {_item} BY {_col}", font_size=10)
    fig.show()

In [ ]:
res.query('RELATIVE_PRIOR_BY_OCC > 0.2')

In [ ]:
_plot_path(res, 'Corn')

In [ ]:
### THIS IS PRETTY MUCH IT

In [ ]:
from sklearn.datasets import load_iris, load_wine

In [ ]:
import pandas as pd
data = load_iris()
print('\n'.join(data['target_names']))

In [ ]:
df = pd.DataFrame(data['data'], columns = data['feature_names'])
df['target'] = data['target']

In [ ]:
df.head()

In [ ]:
##  LET'S SEE OTHERS

In [ ]:
pip install h2o=='3.32.1.2'

In [ ]:
import h2o
h2o.__version__

In [ ]:
import h2o
from h2o.estimators import H2ORuleFitEstimator

In [ ]:
from typing import List, AnyStr

def _replace_str(_source, keys: List, values: (List, AnyStr)):
    equalen = len(keys) == len(values)
    singlevalue = len(values) == 1
    assert sum({equalen, singlevalue}) == 1, f'{FAIL} no apparent mapping between {keys} and {values}'

    for k, v in zip(keys, values if equalen else values * len(keys)):
        _source = _source.replace(k, v)
    return _source

class H2ORule(H2ORuleFitEstimator, object):
    def __init__(self, **kw):
        """ Adjusted to work with pd.DataFrames. """
        h2o.init()
        self.Rules_ = 'tuple()'
        self.frame_ = pd.DataFrame()
        super(H2ORule, self).__init__(**kw)

    @property
    def Rules_(self):
        return self.__Rules_

    @Rules_.setter
    def Rules_(self, _rulette_):
        self.__Rules_ = _rulette_

    @property
    def frame_(self):
        return self.__frame_

    @property
    def TopRule_(self):
        if self.Rules_:
            return self.Rules_[0]
        else:
            print(f'Warning no rules recieved')
            return 'tuple()'

    @frame_.setter
    def frame_(self, _frame_):
        self.__frame_ = _frame_

    @staticmethod
    def _translated(_rl):
        """ Makes the rule pd.DataFrame.query(*_rl) compatible """

        return _replace_str(_rl, keys=['&', ' is NA'], values=['and', ' == 0'])

    @staticmethod
    def _asH2O(_df: pd.DataFrame):
        return h2o.H2OFrame(_df)

    def fit(self, _xx: pd.DataFrame, _yy: pd.Series, **kw):

        _xcols = list(_xx)

        _x = _xx.reset_index().copy(deep=True)
        _x['target'] = _yy

        del _xx
        del _yy

        self.frame_ = _x
        self.train(training_frame=self._asH2O(self.frame_),
                   x=_xcols,
                   y='target')

        self.Rules_ =  list(self.process_rules_to_pd()['Rules'].values)
        
    def process_rules_to_pd(self) -> pd.DataFrame:
        _rule_frame = self._model_json['output']['rule_importance'].as_data_frame()
        return pd.DataFrame(
            _rule_frame.sort_values(
                'coefficient', ascending=False).query(
                'coefficient > 0.0')['rule'].apply(
                lambda r: self._translated(r))).rename(
            columns={'rule': 'Rules'}).query('Rules != ""')

In [ ]:
h2oRule = H2ORule()

In [ ]:
df.head()

In [ ]:
df.columns = list(map(lambda st: st.replace('(', '').replace(')', '').replace(' ', '_'), list(df)))

In [ ]:
df.head()

In [ ]:
features = list(df)[:-1]
target = list(df)[-1]

In [ ]:
df['target'] = df['target'].apply((lambda r: 1 if r != 1 else 0))

In [ ]:
h2oRule.fit(df[features], df[target].apply(lambda r: 1 if r != 1 else 0))

In [ ]:
import seaborn as sns

In [ ]:
h2oRule.Rules_

In [ ]:
sns.scatterplot(data = df, x = 'petal_width_cm', y = 'petal_length_cm', hue = 'target');

In [ ]:
df.query(h2oRule.Rules_[0]).groupby('target')['target'].count()

In [ ]:
!pip install wittgenstein

In [ ]:
import wittgenstein as lw
ripper_clf = lw.RIPPER()

In [ ]:
df

In [ ]:
ripper_clf.fit(df[features], df[target])

In [ ]:
ripper_clf.ruleset_

In [ ]:
ripper_clf.ruleset_.__dict__['rules'][0]

In [ ]:
ripper_clf.ruleset_.__dict__['rules'][1]

In [ ]:
df[df.petal_width_cm.between(1.0,1.3)].groupby('target')['target'].count()

In [ ]:
irep_clf = lw.IREP() # IREP

In [ ]:
irep_clf.fit(df[features], df[target].apply(lambda r: 1 if r == 1 else 0))

In [ ]:
irep_clf.ruleset_

In [ ]:
ripper_clf.ruleset_

In [ ]:
!pip install skope-rules

In [ ]:
from skrules import SkopeRules

In [ ]:
sk = SkopeRules(feature_names = features)

In [ ]:
sk.fit(df[features], df[target])

In [ ]:
df.query(sk.rules_[0][0]).groupby('target')['target'].count()

In [ ]:
df.query(sk.rules_[0][0]).target.value_counts()

### THIS IS PRETTY MUCH IT

In [ ]:
# ### SKOPE RULES
# sk.__dict__

In [ ]:
irep_clf.__dict__['VALID_HYPERPARAMETERS']

In [ ]:
ripper_clf.__dict__['VALID_HYPERPARAMETERS']